# Structuring Python code: Modules and Packages

<img src="doc/python_structure_options.svg" style="width: 800px;"/>


## What is a Python module and what is it good for?

A module is a file consisting of Python code. A module can define functions, classes and variables. A module can also include runnable code.

Use modules to organize your program logically

- Split the code into several files for easier maintenance.
- Group related code into a module.
- Share common code between scripts.
- Publish modules on the web for other people to use (even better: create a **package**, see below).


### Using modules

We have already seen some example of usage in the previous lecture part

```python
import itertools
# Access function from the module
itertools.product

# Alias
import itertools as itools
itool.product

# The following is considered a bad practice
from itertools import *
# Easy to shadow existing variables (also hard for IDEs)
```


## What makes `itertools` into a module and not a package?

It is a single file!
Python has a lot of built-in modules

```python
import sys
print(sys.builtin_module_names)
```


## Creating and using Python modules

Creating own modules in Python is **very simple**:

1. Put any code (variables, functions, classes) that should be part of the module in a Python file.

Let us consider a simple user-defined function, put in the file `user_factorial.py`


In [6]:
import os
os.system("head -n 20 user_factorial.py")

def factorial(n: int) -> int:
    """Return the factorial of n, an exact integer >= 0.

    Args:
       n (int):  n!

    Returns:
       int.  The factorial value::

    >>> factorial(5)
    120
    >>> factorial(0)
    1

    """
    if n == 0:
        return 1
    return n * factorial(n - 1)



0

We can now import this function to our code by calling


In [7]:
from user_factorial import factorial

factorial(10)

3628800

How does Python find your modules? When importing a module (or package module), Python tries to find it in multiples places (in this order):

    - The built-in modules shown above
    - Your current working directory.
    - Paths defined by the environment variable $PYTHONPATH.
    - Some global paths, e.g. /usr/lib/python3.7/site-packages. This depends on your OS and Python installation.

This can be verified as follows via another useful module `sys`


In [8]:
import sys

# Notice the order
sys.path

['/home/dokken/Documents/src/UiO/UiO-IN3110.github.io/lectures/python',
 '/home/dokken/src/mambaforge/envs/UIO-IN3110/lib/python310.zip',
 '/home/dokken/src/mambaforge/envs/UIO-IN3110/lib/python3.10',
 '/home/dokken/src/mambaforge/envs/UIO-IN3110/lib/python3.10/lib-dynload',
 '',
 '/home/dokken/.local/lib/python3.10/site-packages',
 '/home/dokken/src/mambaforge/envs/UIO-IN3110/lib/python3.10/site-packages']

## Test block in a module

Module files can have a test/demo section at the end:

- The block is executed _only if_ the module file is run as a program (not if imported by another script)
- The tests at the end of a module often serve as good examples on the usage of the module

For the problem above, we add a simple hard-coded test of the factorial function


In [22]:
import os
os.system("tail -n 7 user_factorial.py")

if __name__ == "__main__":
    import math, sys
    N = int(sys.argv[1])
    print(f"Testing user defined factorial function for {N=}")
    user_n = factorial(N)
    ref_factorial = math.factorial(N)
    assert user_n == math.factorial(N), f"Factorial function returning wrong answer {user_n}!={ref_factorial}"

0

In [23]:
import subprocess
subprocess.run(["python3", "user_factorial.py", "6"])

Testing user defined factorial function for N=6


CompletedProcess(args=['python3', 'user_factorial.py', '6'], returncode=0)

## What is a package?

A package is a hierarchical file directory structure that consists of modules and subpackages and sub-subpackages, and so on.

**Example:**

```python
from scipy.optimize import minimize
#      ^      ^               ^
#      |      |               |
#   Package   |               |
#           Module            |
#                          Function
```

Packages allow to organize modules and scripts into single environment. These can then easily be distributed and imported by name.


Python comes with a set of powerful packages, e.g.

- **scipy** Scientific Python
- **numpy** Numerical Python
- **ipython** Interactive Python
- **matplotlib** Plotting
- **pandas** Data analysis
- **scikit learn** Machine learning

_Several useful packages are included in Python distributions like Anaconda_


## Creating a package

- A set of modules can be collected in a _package_
- A package is organized as module files in a directory tree
- Each subdirectory must have a `__init__.py` file (can be empty)
- More infos: [Section 6 in the Python Tutorial](https://docs.python.org/3/tutorial/modules.html)

We have a sample package in the `data/my-package` directory. The package tree is as follows


In [ ]:
!tree data/my-package

## Using a package

One option is to manipulate Bash variable PYTHONPATH

```bash
> export PYTHONPATH=/path/to/my-package:$PYTHONPATH
```

Note: The search path will be lost when you open a new Bash session. We can accomplish the same by manipulating the
path from within python


In [ ]:
import os
import sys

pkg_path = os.path.abspath("./data/my-package")

sys.path.append(pkg_path)
sys.path

In [ ]:
from pkg.printing import print_red

print_red("A RED MESSSAGE")

A better option is to reate a `setup.py` file in your package root directory.

```python
from setuptools import setup

setup(name = 'pkg',
      version = '0.1',
      description = 'Simple package',
      author = 'Miroslav Kuchta',
      author_email = 'miroslav.kuchta@gmail.com',
      # url = 'https://github.com/mirok/...',
      packages = ['pkg'],
      package_dir = {'pkg': 'pkg'}
)

```


and install with one of

```bash
pip3 install . --user  # For single-user installation
pip3 install .         # For system wide installation (requires root)
pip3 install --editable .        # For developer mode (changes to source are immediately reflected)
```

you can remove the package installation again with:

```bash
pip3 uninstall pkg
```


We will now work on the package with an aim to cover some of the existing functionality by testing and add new functionality by
practicing test-drive-development. First some crash-course in Python unit testing


## Testing

### Why should we test?

- To check correctness of software.
- To ensure that future changes do not break functionality.
- To check if the software runs succesfully in a different environment (newer Python version, upgraded libraries, different operating system)

### Unit testing

1. Identify a _unit_ in your program that should have a well defined behavior given a certain input. A unit can be a:
   - function
   - module
   - entire program
2. Write a test function that calls this input and checks that the output/behavior is as expected
3. The more the better! Preferably on several levels (function/module/program).


## Unit testing in Python

Use a test framework like [py.test](http://docs.pytest.org/en/latest/). Several other frameworks exist as well.

```bash
pip3 install pytest
```

Call `py.test` in the folder containing your project. The tools will look for anything that looks like a test (e.g. `test_*()` functions in `test_*.py` files) in your project (also subdirectories).


```python
# From test_factorial.py
# Test of factorial function from module
from factorial_doctest_exceptions import factorial
import pytest


def test_correctness():
    assert factorial(0) == 1
    assert factorial(1) == 1

# We can generate more tests with decorators
@pytest.mark.parametrize('inp, out', [(2, 2), (3, 6)])
def test_correctness_dec(inp, out):
    assert factorial(inp) == out


def test_raises_not_int():
    with pytest.raises(AssertionError):
        factorial('1')


def test_raises_negative():
    with pytest.raises(ValueError):
        factorial(-1)


@pytest.mark.xfail
def test_expected_fail():
    assert factorial(3) == 4

def test_will_fail():
    assert factorial(5) == 6
```

We can run all the tests or be zoom on individual functions


In [ ]:
!pytest .

In [ ]:
!pytest test_factorial.py::test_raises_negative

Note that you can use pytest to run the docstring tests also, for example

```bash
pytest --doctest-modules factorial_doctest_exceptions.py
```


We are now ready to add test suite to our package. It is customary to write them in a test folder. Let us create it


In [ ]:
import os

not os.path.exists("./data/my-package/test") and os.makedirs("./data/my-package/test")

The scanario for class demo is as follows:

1.  Add tests to cover function as is
2.  Improve the function and cover the functionality under test suit
3.  Add new functionality
4.  Illustrate debugging in the process
    For debugging we will use [pdb](https://docs.python.org/3/library/pdb.html), `conda install pdb`, or IPython's embed.

```python
from IPython import embed
embed()
```


Of course, the ultimate form of debuggin are `print` statements :)
